In [12]:
from pcl_lib import Msger
import numpy as np
import math

In [2]:
pcl_frames = []
for i in range(50):
    pcl_frames.append(np.loadtxt('../percpt/pcls/pcl_{0}.out'.format(i)))

In [4]:
msger = Msger()
for pcl in pcl_frames:
    msger.publish_pcl_msg(pcl)

[INFO] [1525207434.665351]: Initializing pcl2 publisher node...
[INFO] [1525207435.805245]: publishing sample pointcloud.. !
[INFO] [1525207436.040865]: publishing sample pointcloud.. !
[INFO] [1525207436.253948]: publishing sample pointcloud.. !
[INFO] [1525207436.511496]: publishing sample pointcloud.. !
[INFO] [1525207436.753558]: publishing sample pointcloud.. !
[INFO] [1525207436.993992]: publishing sample pointcloud.. !
[INFO] [1525207437.235629]: publishing sample pointcloud.. !
[INFO] [1525207437.476592]: publishing sample pointcloud.. !
[INFO] [1525207437.717935]: publishing sample pointcloud.. !
[INFO] [1525207437.959157]: publishing sample pointcloud.. !
[INFO] [1525207438.200885]: publishing sample pointcloud.. !
[INFO] [1525207438.440669]: publishing sample pointcloud.. !
[INFO] [1525207438.676541]: publishing sample pointcloud.. !
[INFO] [1525207438.920959]: publishing sample pointcloud.. !
[INFO] [1525207439.153288]: publishing sample pointcloud.. !
[INFO] [1525207439.38

In [5]:
def F2I(val,ori=60,scale=512.0/120):
    import math
    return int(math.floor((ori-val)*scale))
def I2F(val,ori=60,scale=2*60.0/512):
    import math
    return -int(val*scale-ori)
def Sphe2Carte(vr,hr,R):
    x = R*np.cos(vr) * np.cos(hr)
    y = R * np.cos(vr)*np.sin(hr)
    z = R * np.sin(vr)
    return [x,y,z]
def Cart2Sphe(x,y,z):
    import math
    r = np.linalg.norm([x,y,z])
    vr = math.atan2(z/r,np.linalg.norm([x/r,y/r]))
    hr = math.atan2(y/r,x/r)
    return [vr,hr,r]
def Sphe2CartIdx(vr,hr,R):
    x,y,z = Sphe2Carte(vr,hr,R)
    i = F2I(x)
    j = F2I(y)
    return [i,j]

In [6]:
# prepare data
adv_features = np.load('../percpt/adv_features/adv_feature_0.npy')
adv_features = np.swapaxes(adv_features,0,3)[:,:,:,0]

features = np.loadtxt('../percpt/features/features_0.out')
features = np.reshape(features,(8,512,512))

In [7]:
# lidar config
res_deg = 0.05
h_range = int(np.ceil(360 / res_deg)) 
v_range = 64

v_rs = 40./v_range

pcl_polar = np.zeros([h_range,v_range])

In [8]:
# pcl in polar frames
v_list = []
h_list = []
for p in pcl_frames[0]:
    p_polar = Cart2Sphe(p[0],p[1],p[2])
#     h_list.append(np.floor((p_polar[1] + np.pi)*180/np.pi/res_deg))
    h_idx = np.floor((p_polar[1] + np.pi)*180/np.pi/res_deg)
#     v_list.append(np.floor((p_polar[0]*180/np.pi + 40)/v_rs))
    v_idx = np.floor((p_polar[0]*180/np.pi + 35)/v_rs)
    v_idx = int(np.max([np.min([v_idx,v_range-1]),0]))
    h_idx = int(np.max([np.min([h_idx,h_range-1]),0]))
#     print p_polar[2],v_idx,h_idx
#     if pcl_polar[h_idx,v_idx] != 0:
#         print p_polar[2], pcl_polar[h_idx,v_idx]
    pcl_polar[h_idx,v_idx] = p_polar[2]

In [9]:
# original pcl
pcl = []
for v_idx in range(v_range):
    for h_idx in range(h_range):
        v = (v_idx * v_rs - 35)/180.0 * np.pi
        h = h_idx * res_deg /180. * np.pi - np.pi
        if pcl_polar[h_idx,v_idx] != 0:
            pcl.append(Sphe2Carte(v,h,pcl_polar[h_idx,v_idx]))

print len(pcl)
            
msger.publish_pcl_msg(pcl)

94034
[INFO] [1525207476.067225]: publishing sample pointcloud.. !


In [13]:
def SynthesisPolarPCLAdv(features, adv_features, pcl_polar):
    adv_cnt = np.floor(np.exp(adv_features[2]))-1
    cnt = np.floor(np.exp(features[2]))-1
    
    print "Real cnt: ",np.sum(cnt)
    print "Expected cnt: ",np.sum(adv_cnt)
    
    adv_pcl_polar = np.zeros([h_range,v_range])
    
    # adding point from lower height
    # adding point from original pcl
    for v_idx in range(v_range):
        for h_idx in range(h_range):
            v = (v_idx * v_rs - 35)/180.0 * np.pi
            h = h_idx * res_deg /180. * np.pi - np.pi
            if pcl_polar[h_idx,v_idx] != 0:
                p = Sphe2Carte(v,h,pcl_polar[h_idx,v_idx])
                i = F2I(p[0])
                j = F2I(p[1])
                if i < 0 or i > 511 or j < 0 or j > 511:
                    continue
                if adv_cnt[i,j] > 0:
                    adv_cnt[i,j] = adv_cnt[i,j] - 1
                    adv_pcl_polar[h_idx,v_idx] = np.copy(pcl_polar[h_idx,v_idx])
    # filling additional point
    added_pts = 0
    for i in range(512):
        for j in range(512):
            if adv_cnt[i,j] > 0:
                y_range = [I2F(j),I2F(j+1)]
                x_range = [I2F(i),I2F(i+1)]
                h_all = [math.atan2(y_range[0],x_range[0]),math.atan2(y_range[0],x_range[1])]
                h_r =  [np.min(h_all),np.max(h_all)]
                h_idx_r = [np.floor((h + np.pi)*180/np.pi/res_deg) for h in h_r ]
                for h in range(int(h_idx_r[0]),int(h_idx_r[1])):
                    for v in range(v_range):
                        if adv_pcl_polar[h,v_range - 1 - v] == 0:
                            for eps in range(-5,5):
                                if h+eps >=0 and h+eps <h_range and eps != 0 and pcl_polar[h+eps,v_range - 1 - v] != 0:
                                    adv_pcl_polar[h,v_range - 1 - v] = np.copy(pcl_polar[h+eps,v_range - 1 - v])
                                    adv_cnt[i,j] -= 1
                                    added_pts += 1
                                    break
                        if adv_cnt[i,j] <= 0:
                            break
                    if adv_cnt[i,j] <= 0:
                        break
#                 if adv_cnt[i,j] > 0:
#                     print "Unresolved adv cnt: ", adv_cnt[i,j]
    print "Adding pts: ", added_pts
    
    pcl = []
    for v_idx in range(v_range):
        for h_idx in range(h_range):
            v = (v_idx * v_rs - 35)/180.0 * np.pi
            h = h_idx * res_deg /180. * np.pi - np.pi
            if adv_pcl_polar[h_idx,v_idx] != 0:
                pcl.append(Sphe2Carte(v,h,adv_pcl_polar[h_idx,v_idx]))

    print len(pcl)

    msger.publish_pcl_msg(pcl)
    return adv_pcl_polar

In [14]:
adv_pcl_polar = SynthesisPolarPCLAdv(features,adv_features,pcl_polar)

Real cnt:  85505.0
Expected cnt:  91346.0
Adding pts:  1965
83720
[INFO] [1525207512.675500]: publishing sample pointcloud.. !


In [15]:
# adv pcl
pcl = []
for v_idx in range(v_range):
    for h_idx in range(h_range):
        v = (v_idx * v_rs - 35)/180.0 * np.pi
        h = h_idx * res_deg /180. * np.pi - np.pi
        if adv_pcl_polar[h_idx,v_idx] != 0:
            pcl.append(Sphe2Carte(v,h,adv_pcl_polar[h_idx,v_idx]))

print len(pcl)
            
msger.publish_pcl_msg(pcl)

83720
[INFO] [1525207520.025033]: publishing sample pointcloud.. !


In [16]:
msger.publish_pcl_msg(pcl_frames[0])

[INFO] [1525207524.723043]: publishing sample pointcloud.. !
